1. Parameter Accuracy

This part will check how accurate the LLM parse the parameters for the function call after reading the query.

In [2]:
import json
import re

In [3]:
def paraacc(log,eval):
    #attractions
    att_deno = 2
    att_num = 2
    for step in reversed(log):
        if step['state'] == 'Successful':
            if "AttractionSearch" in step['action']:
                full = str(step['action'])
                pattern = r'\[.*\]'
                match = re.search(pattern, full)
                if(match):
                    preferences = match.group(0).lower()
                    #print(preferences)
                
                pattern2 = r'\[(.*),\s*\[(.*)\]\]'
                match = re.search(pattern2, preferences)
                if match:
                    budget = match.group(1).strip()
                    pref = match.group(2).strip()
                    #print(budget, pref)
                    pref_list = pref.split(', ')
                    #print(pref_list)
                if budget != eval['price'][0].lower():
                    att_num -= 1
                for p in pref_list:
                    if p not in eval['attraction']:
                        att_num -= 1

    #accommodation
    hotel_deno = 0
    hotel_num = 0
    for step in reversed(log):
        if step['state'] == 'Successful':
            if "AccommodationSearch" in step['action']:
                full = str(step['action'])
                pattern = r'\[.*\]'
                match = re.search(pattern, full)
                if(match):
                    preferences = match.group(0).lower()
                    #print(preferences)
                
                pattern2 = r'\[(.*),\s*\[(.*)\]\]'
                match = re.search(pattern2, preferences)
                if match:
                    budget = match.group(1).strip()
                    pref = match.group(2).strip()
                    #print(budget, pref)
                    pref_list = pref.split(', ')
                    #print(pref_list)
                    hotel_deno = 1 + len(pref_list)
                    hotel_num = hotel_deno
                    #print(hotel_deno)
                if budget != eval['price'][0].lower():
                    hotel_num -= 1
                for p in pref_list:
                    if p not in eval['hotel']:
                        hotel_num -= 1

    #restaurant
    res_deno = 0
    res_num = 0
    for step in reversed(log):
        if step['state'] == 'Successful':
            if "RestaurantSearch" in step['action']:
                full = str(step['action'])
                full = 'Action 3: RestaurantSearch[Moderate Budget, Vietnamese, [Good Flavor, Good Value]]'
                pattern = r'\[.*\]'
                match = re.search(pattern, full)
                if(match):
                    preferences = match.group(0).lower()
                    #print(preferences)
                
                pattern2 = r'\[(.*),(.*),\s*\[(.*)\]\]'
                match = re.search(pattern2, preferences)
                if match:
                    budget = match.group(1).strip()
                    cuisine = match.group(2).strip()
                    pref = match.group(3).strip()
                    #print(budget, cuisine, pref)
                    pref_list = pref.split(', ')
                    #print(pref_list)
                    res_deno = 2 + len(pref_list)
                    res_num = res_deno
                    #print(res_deno)
                if budget != eval['price'][0].lower():
                    res_num -= 1
                if cuisine != eval['cuisine'][0].lower():
                    res_num -= 1
                for p in pref_list:
                    if p not in eval['restaurants']:
                        res_num -= 1


    
    return res_num + att_num + hotel_num , res_deno + att_deno + hotel_deno

def getParameterACC(model,numPlan):
    allnum = 0
    alldeno = 0
    with open (f'Outputs/plans/{model}/toolUseLogs.jsonl', 'r') as f:
        logs = [json.loads(line.strip()) for line in f]
    with open (f'Prompts/evals.jsonl', 'r') as f:
        evals = [json.loads(line.strip()) for line in f]
    
    for log,eval in zip(logs,evals):
        num,deno = paraacc(log,eval)
        allnum += num
        alldeno += deno

    rate = allnum/alldeno
    return rate

In [4]:
getParameterACC('llama318b',10)

FileNotFoundError: [Errno 2] No such file or directory: 'Outputs/plans/llama318b/toolUseLogs.jsonl'